In [55]:
#imports
library(readr)
suppressMessages(library(tigerstats))
suppressMessages(library(ggplot2))
suppressMessages(library(gridExtra))
suppressMessages(library("DESeq2"))
suppressMessages(library("vsn"))
suppressMessages(library("pheatmap"))
suppressMessages(library("RColorBrewer"))
suppressMessages(library(repr))

In [2]:
mc <- suppressMessages(read_tsv("morphological-counts.tsv"))
rna <- suppressMessages(read_tsv("RNA-read-counts.tsv"))
reactome <- suppressMessages(read_tsv("c2.cp.reactome.v7.5.1.symbols.gmt"))

Warning message:
"One or more parsing issues, see `problems()` for details"


In [3]:
#normalization (we use frequency instead of count)
for (i in 1:136){
    mc[i,2:65] <- mc[i,2:65]/sum(mc[i,2:65])
} 


In [20]:
#filtering the data (only take the genes in the top quartile of expression)
means <- rowMeans(rna[,3:138])
#med <- median(means)
qt <- quantile(means)[4]
cond <- means > qt
rna$cond <- cond
rna <- rna[rna$cond == TRUE,]
rna

Name,Description,GTEX-11EM3-0126,GTEX-11EQ9-0626,GTEX-11LCK-0526,GTEX-11NSD-0126,GTEX-11O72-2326,GTEX-11P7K-0226,GTEX-11P81-0126,GTEX-11P82-0226,...,GTEX-1QPFJ-0626,GTEX-1QW4Y-0226,GTEX-1R9JW-0226,GTEX-1R9PM-0226,GTEX-1RAZA-0826,GTEX-1RAZQ-0926,GTEX-1RAZR-1126,GTEX-1S5ZU-0726,GTEX-1S83E-0726,cond
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
ENSG00000227232.5,WASH7P,122,174,85,115,197,263,119,125,...,183,124,74,191,251,246,91,169,162,TRUE
ENSG00000279457.4,RP11-34P13.18,241,388,84,370,368,156,303,141,...,291,422,336,288,366,267,235,240,454,TRUE
ENSG00000225972.1,MTND1P23,142,120,76,125,45,21804,41,70,...,29,29,25,20,44,59,28,38,42,TRUE
ENSG00000225630.1,MTND2P28,12884,22597,9366,9301,19615,7169,9587,18815,...,14655,10444,6079,7345,16137,12594,7127,7727,10520,TRUE
ENSG00000237973.1,MTCO1P12,3138,3534,1719,3210,405,7076,570,3186,...,1384,1431,1034,2473,2497,1311,835,390,1730,TRUE
ENSG00000229344.1,MTCO2P12,470,537,255,934,159,361,107,230,...,86,95,119,66,179,131,33,98,177,TRUE
ENSG00000248527.1,MTATP6P1,52329,78285,34684,34299,24116,38504,26788,69837,...,30880,34259,30430,18410,56146,45142,19862,23143,39812,TRUE
ENSG00000198744.5,MTCO3P12,436,782,253,1051,301,329,141,1259,...,81,72,322,57,155,139,89,105,160,TRUE
ENSG00000228327.3,RP11-206L10.2,718,682,494,434,582,569,471,375,...,610,462,295,422,599,673,339,484,840,TRUE


In [53]:
#change the structure of the dataset to have the gene names as the row names
dataset = data.frame(rna[, 3:138])
row.names(dataset) <- t(rna[1:14050,1])
dataset

,GTEX.11EM3.0126,GTEX.11EQ9.0626,GTEX.11LCK.0526,GTEX.11NSD.0126,GTEX.11O72.2326,GTEX.11P7K.0226,GTEX.11P81.0126,GTEX.11P82.0226,GTEX.11TT1.1126,GTEX.11VI4.0226,...,GTEX.1QP9N.0126,GTEX.1QPFJ.0626,GTEX.1QW4Y.0226,GTEX.1R9JW.0226,GTEX.1R9PM.0226,GTEX.1RAZA.0826,GTEX.1RAZQ.0926,GTEX.1RAZR.1126,GTEX.1S5ZU.0726,GTEX.1S83E.0726
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000227232.5,122,174,85,115,197,263,119,125,103,135,...,111,183,124,74,191,251,246,91,169,162
ENSG00000279457.4,241,388,84,370,368,156,303,141,411,297,...,590,291,422,336,288,366,267,235,240,454
ENSG00000225972.1,142,120,76,125,45,21804,41,70,72,28,...,38,29,29,25,20,44,59,28,38,42
ENSG00000225630.1,12884,22597,9366,9301,19615,7169,9587,18815,13056,7859,...,6884,14655,10444,6079,7345,16137,12594,7127,7727,10520
ENSG00000237973.1,3138,3534,1719,3210,405,7076,570,3186,3285,1711,...,1918,1384,1431,1034,2473,2497,1311,835,390,1730
ENSG00000229344.1,470,537,255,934,159,361,107,230,221,116,...,134,86,95,119,66,179,131,33,98,177
ENSG00000248527.1,52329,78285,34684,34299,24116,38504,26788,69837,66201,29905,...,48466,30880,34259,30430,18410,56146,45142,19862,23143,39812
ENSG00000198744.5,436,782,253,1051,301,329,141,1259,226,131,...,112,81,72,322,57,155,139,89,105,160
ENSG00000228327.3,718,682,494,434,582,569,471,375,500,441,...,625,610,462,295,422,599,673,339,484,840


In [54]:
#test of differential expression computation with cluster 4
dds = DESeqDataSetFromMatrix(dataset,colData=mc, design = ~ Mophological.cluster.4)
dds <- DESeq(dds)
dds <- DESeq(dds, quiet=TRUE)
print(resultsNames(dds))
res <- results(dds)
res.05 <- results(dds, alpha = 0.05)
resSig <- subset(res, padj < 0.05)
print(rownames(resSig))


converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Intercept"              "Mophological.cluster.4"
   [1] "ENSG00000237973.1"  "ENSG00000228327.3"  "ENSG00000225880.5" 
   [4] "ENSG00000162572.20" "ENSG00000169972.11" "ENSG00000149527.17"
   [7] "ENSG00000238164.6"  "ENSG00000157873.17" "ENSG00000130764.9" 
  [10] "ENSG00000069424.14" "ENSG00000116237.15" "ENSG00000215788.9" 
  [13] "ENSG00000162426.14" "ENSG00000162441.11" "ENSG00000173614.13"
  [16] "ENSG00000162444.11" "ENSG00000175206.10" "ENSG00000028137.18"
  [19] "ENSG00000184908.17" "ENSG00000142632.16" "ENSG00000117118.9" 
  [22] "ENSG00000179023.8"  "ENSG00000162543.5"  "ENSG00000090432.6" 
  [25] "ENSG00000169641.13" "ENSG00000188529.14" "ENSG00000020633.18"
  [28] "ENSG00000169442.8"  "ENSG00000204160.11" "ENSG00000142784.15"
  [31] "ENSG00000117751.17" "ENSG00000130775.15" "ENSG00000130772.13"
  [34] "ENSG00000197989.13" "ENSG00000198492.15" "ENSG00000116350.16"
  [37] "ENSG00000182866.16" "ENSG00000121903.14" "ENSG00000092850.11"
  [40] "ENSG00000116871.15" "ENSG000

In [60]:
resSig

log2 fold change (MLE): Mophological.cluster.4 
Wald test p-value: Mophological.cluster.4 
DataFrame with 1702 rows and 6 columns
                    baseMean log2FoldChange     lfcSE      stat      pvalue
                   <numeric>      <numeric> <numeric> <numeric>   <numeric>
ENSG00000237973.1   3216.798      -30.36948   6.61011  -4.59440 4.33993e-06
ENSG00000228327.3    578.000        6.59688   2.36875   2.78496 5.35351e-03
ENSG00000225880.5    218.198        7.82493   2.59833   3.01152 2.59946e-03
ENSG00000162572.20   886.218        8.01882   2.70209   2.96764 3.00099e-03
ENSG00000169972.11   455.993       -3.94406   1.36385  -2.89187 3.82959e-03
...                      ...            ...       ...       ...         ...
ENSG00000203879.11  10111.95        3.53044  0.955556   3.69465 2.20192e-04
ENSG00000160219.11   1075.12        6.11096  1.830561   3.33830 8.42934e-04
ENSG00000185010.13   2095.40      -10.14870  2.558029  -3.96739 7.26636e-05
ENSG00000198804.2  994255.04      

In [61]:
res

log2 fold change (MLE): Mophological.cluster.4 
Wald test p-value: Mophological.cluster.4 
DataFrame with 14050 rows and 6 columns
                   baseMean log2FoldChange     lfcSE       stat      pvalue
                  <numeric>      <numeric> <numeric>  <numeric>   <numeric>
ENSG00000227232.5   188.322       2.415753   2.98346  0.8097145 4.18104e-01
ENSG00000279457.4   360.692       3.288385   2.32433  1.4147665 1.57137e-01
ENSG00000225972.1  1673.862     -30.000000  12.66222 -2.3692536 1.78240e-02
ENSG00000225630.1 13771.356       0.251998   3.50657  0.0718646 9.42710e-01
ENSG00000237973.1  3216.798     -30.369483   6.61011 -4.5944017 4.33993e-06
...                     ...            ...       ...        ...         ...
ENSG00000212907.2   99343.9       -4.36318   2.90978   -1.49949  0.13374654
ENSG00000198886.2  969909.5       -3.81706   2.92771   -1.30377  0.19231242
ENSG00000198786.2  198308.1       -9.77217   4.56354   -2.14136  0.03224523
ENSG00000198695.2   59198.3      

In [56]:
length(rownames(resSig))

[1] 1702

In [ ]:
#fonction pour remplir la ligne du tableau 'assoc_table' associée avec une variable clinique numérique
num_dif_exp <- function(condition){
    cond <- as.formula(paste("~", condition))
    print(cond)
    dds = DESeqDataSetFromMatrix(rna_test[, 3:138],colData=mc, design = cond)
    dds <- DESeq(dds, quiet=TRUE)
    print(resultsNames(dds))
    res <- results(dds)
    res.05 <- results(dds, alpha = 0.05)
    resSig <- subset(res, padj < 0.05)
    print(rownames(resSig))
    #assoc_table[condition,] <<- colnames(assoc_table) %in% rownames(resSig)
}

In [58]:
clusters = colnames(mc)[2:65]

In [59]:
print(clusters)

 [1] "Mophological.cluster.0"  "Mophological.cluster.1" 
 [3] "Mophological.cluster.2"  "Mophological.cluster.3" 
 [5] "Mophological.cluster.4"  "Mophological.cluster.5" 
 [7] "Mophological.cluster.6"  "Mophological.cluster.7" 
 [9] "Mophological.cluster.8"  "Mophological.cluster.9" 
[11] "Mophological.cluster.10" "Mophological.cluster.11"
[13] "Mophological.cluster.12" "Mophological.cluster.13"
[15] "Mophological.cluster.14" "Mophological.cluster.15"
[17] "Mophological.cluster.16" "Mophological.cluster.17"
[19] "Mophological.cluster.18" "Mophological.cluster.19"
[21] "Mophological.cluster.20" "Mophological.cluster.21"
[23] "Mophological.cluster.22" "Mophological.cluster.23"
[25] "Mophological.cluster.24" "Mophological.cluster.25"
[27] "Mophological.cluster.26" "Mophological.cluster.27"
[29] "Mophological.cluster.28" "Mophological.cluster.29"
[31] "Mophological.cluster.30" "Mophological.cluster.31"
[33] "Mophological.cluster.32" "Mophological.cluster.33"
[35] "Mophological.cluster.34" 

In [ ]:
for(variable in clusters){
    num_dif_exp(variable)}

In [ ]:
assoc_table